In [33]:
# use Computer Hardware Data Set link http://archive.ics.uci.edu/ml/datasets/Computer+Hardware 
# related papaers 《learning with continuous vlaues》
import pandas as pd
import numpy as np


In [14]:
names = ['vendor_name','model_name','myct','mmin','mmax','cach','chmin','chmax','prp','erp']

In [59]:
frame = pd.read_csv('./data/data.csv',header = None,names = names ,usecols = names[2:-1])

In [67]:
X = frame[names[2:-2]]
y = frame[names[-2]]

In [69]:
print(X.columns)
print(y.shape)

Index(['myct', 'mmin', 'mmax', 'cach', 'chmin', 'chmax'], dtype='object')
(209,)


In [70]:
cols = names[2:]
sd = np.std(frame['prp'].values)

![选择属性的条件](img/error.png)

- 这里最大化 error
- 和 CART 不同的是，CART使用的标准差或者偏差

## 先回顾一下线性回归
- 二元线性回归使用最小二乘法来算损失函数,得到参数的值为
$$\begin{split}
a &= \frac{\sum_{i}^{m}{x_{i}y_{i}-m\bar{x}\bar{y}}}{\sum_{i}^{m}x_{i}^{2}-m\bar{x}^{2}} \\ 
b &= \bar{y} - a\bar{x}
\end{split}
$$
- 评判标准
    * 均方误差(MSE)
        ![mse](img\mse.png)
    * 根均方误差(RMSE),其是均方误差的开方
    * 平均绝对误差(MAE)
        ![mae](img\mae.png)
    * R-square 
        ![r-square](img\R-square.png)
- 多元线性回归求解
    * 可以使用矩阵求导一步求得
        ![multivariate_linear_model](img\multivariate_linear_model.png)

### M5 model tree 特点
- 在每一个点使用标准回归技术建立一个多元回归模型
- 使用(n+v)/(n-v)进行模型的平均残差进行修正。n是训练，v是模型参数的个数
- 在每个叶子节点对线性模型进行简化，此处使用的是贪心的方法。(此处使用的是测试集)
- 对每一个非节点进行剪枝判断，根据评估误差，来决定该节点的子节点是一个叶子还是不变(后剪枝)。
- 对模型预测到的值进行从叶子往上进行一个平滑( 这个应该是对未知数据的)
    * 平滑方程
        ![smooth_function](img\smooth.png)
        其中PV($S_{i}$)是在$S_{i}$处的预测值$n_{i}$是包含的测试数据总数。
        
- 使用(1) relative error来做为而性能的比较，就是1 - $R^2$ (2) correlation 是指预测值和真实值的相关性 (3) 是percentage deviation 残差和目标比值的平均(不适用于目标为0的情况)。


In [230]:
## 先使用决策树看一下效果
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [231]:
# 自定义分数值
def score(reg, X, y):
    pred_y = reg.predict(X)
    # 这里应该是取绝对值
    return np.mean(abs(y - pred_y) / y) * 100

In [205]:
regressor = DecisionTreeRegressor(criterion = 'mse',max_depth = 4)
score = cross_val_score(regressor,X,y,scoring = score,cv = 10)

In [206]:
np.min(score)

36.975796268630475

In [302]:
# 线性回归的效果
from sklearn.linear_model import LinearRegression
from numpy.linalg import *
lr = LinearRegression()
score = cross_val_score(lr,X,y,scoring = score,cv = 10)

In [203]:
np.min(score)

46.805321430605126

In [363]:
# just try
# 只用深度来进行预剪枝，只在叶子中训练一个多元线性模型，不进行后剪枝，而且没有对模型进行属性选择，不对其进行平滑，不进行残差修正
class Node(object):
    def __init__(self,w,split_feature,split_value,is_leaf):
        self.w = None
        self.split_index = split_feature # 划分特征的所以
        self.split_value = split_value # 划分值
        self.lef = None
        self.rig = None
        self.is_leaf = is_leaf
    def set(self,is_leaf,w):
        self.is_leaf = is_leaf
        self.w = w
        
class simpleM5(object):
    def __init__(self,max_depth):
        self.max_depth = max_depth
    def fit(self,train_data,train_label):
        self.n = train_data.shape[0]
        self.m = train_data.shape[1]
        self.attri = np.array(range(train_data.shape[1]))
        self.root = self._build_tree(0,train_data.values,train_label.values)
    def _build_tree(self, dep,data,label): #（np.ndarrays,np.ndarrays) 
        # 在该层确定划分的属性索引，还有属性值
        if(dep > self.max_depth):
            return None # 当达到最大深度时返回
        # 计算train_label的标准差，找到使得切分后，左右子空间标准差的期望 最小的最佳切分点
        mi_index = -1;
        mi_value = 1e9
        mi_index_value = -1
        for col in self.attri:
            data_col = data[:,col]
            sort_index = np.argsort(data_col)
            data_col_sort = data_col[sort_index]
            label_sort = label[sort_index]
            for value in (np.unique(data_col_sort)):
                index = data_col_sort < value
                less_data = data_col_sort[index]
                less_label = label_sort[index]
                tmp = len(less_data) / self.n * np.std(less_label)
                
                index = data_col_sort >= value
                more_data = data_col_sort[index]
                more_label = label_sort[index]
                tmp += len(more_label) / self.n * np.std(more_label)
                if(tmp < mi_value):
                    mi_value = tmp
                    mi_index = col
                    mi_index_value = value
        # 得到划分点，将数据集按照最佳划分点进行划分
        node = Node(None,mi_index,mi_index_value,False)
        index = data[:,mi_index] < mi_index_value
        left_data = data[index]
        left_label = label[index]
        index = data[:,mi_index] >= mi_index_value
        right_data = data[index]
        right_label = label[index]
        tmp = dep + 1
        node.left = self._build_tree(tmp,left_data,left_label)
        node.right = self._build_tree(tmp, right_data,right_label)
        if node.left == None and node.right == None: # 是叶子节点
            n = data.shape[0]
            set_val = set()
            while(len(set_val) < n):
                x = np.random.randint(0,self.m,1)
                if x[0] not in set_val:
                    set_val.add(x[0])
            data = data[:,list(set_val)]
            print(data.shape)
            node.set(np.dot(np.dot(inv(np.dot(data.transpose(), data)), data.transpose()), label),True) # 一步算法多元回归的的参数
        return node
    def predict(self,test):
        result = []
        root = self.root
        for t in test:
           result.append(self._dfs(root,t))
        return np.array(result)

    def _dfs(self,root,test):
        if root.is_leaf:
            return np.dot(root.w,test)
        if test[root.split_index] < root.split_value:
            return self._dfs(root.left,test)
        else:
            return self._dfs(root.right,test) #important

In [364]:
sm = simpleM5(max_depth = 3)
from sklearn.model_selection import KFold

In [365]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
y.shape

(209,)

In [366]:
scores = []
for train_index,test_index in kfold.split(X):
    print('hello')
#     print((train_index).shape,(test_index).shape)
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y[train_index],y[test_index]
    m5 = simpleM5(max_depth = 3)
    m5.fit(X_train, y_train)
    scores.append(score(m5,y_test,y_pred))

hello
hh
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
hh
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
her

/Users/icdi/Desktop/py_ws/venv/lib/python3.5/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/icdi/Desktop/py_ws/venv/lib/python3.5/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/icdi/Desktop/py_ws/venv/lib/python3.5/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
ther

there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
ther

there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
ther

...

there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
ther

there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
ther

there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
ther

there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there
there


KeyboardInterrupt: 